In [111]:
import sys
sys.executable

'/home/goonieb/gitrep/bandstour/venv/bin/python'

In [112]:
import os
import json
import pymongo
import musicbrainzngs
import arrow
import re
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict, OrderedDict

from dateutil import parser
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [113]:
###BEFORE RUNNING THIS ONE YOU SHOULD CLEAN YOUR MONGO DB OF artist strings starting with &, with + and such, or remove the JSon files first"
#example :db.getCollection('selectedArtists').find({"_id": /&.*/})
#this needs to be done by hand

##Variables
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
# setup mongo
client = MongoClient()
db = client["bandstour"]
venues =  db["venues"]
artists =  db["selectedArtists"]
FUZZ_RATIO = 90
NB_remplacements = 0
##input function
##https://gist.github.com/hrouault/1358474
def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.
    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).
    The "answer" return value is one of "yes" or "no".
    """
    valid = {"yes":True,   "y":True,  "ye":True,
             "no":False,     "n":False}
    if default == None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "\
"(or 'y' or 'n').\n")
#Clean a string with regex
def slugify(s):
    """
    Simplifies ugly strings into something URL-friendly.
    >>> print slugify("[Some] _ Article's Title--")
    some-articles-title
    """

    s = re.sub('\s+', ' ', s)
    s = s.replace(' ', '-')

    s = re.sub('\W+', ' ', s)

    s = s.strip()

    s = s.lstrip()
    s = re.sub('\W+', '_', s)
    return s

#WE WANT TO LOOP OVER ARTISTS IN ORDER TO DEDUPLICATE THE RECORDS AND ASSIGN TOURS TO ONE ENTITY ONLY.
#ex: "   dede" =add his dates to> "dede" who already has his.
#in case there is an ambiguity, we will have to choose, at the end
replacement_list =[]
print "  _____________________________"
print " |_couples_d'ARTISTES_de_dups__|"
print " |_NOM_|_SLUG_|nb_dates_|_mbid_|"

  _____________________________
 |_couples_d'ARTISTES_de_dups__|
 |_NOM_|_SLUG_|nb_dates_|_mbid_|


In [ ]:
for artistA in artists.find({},no_cursor_timeout=True) :
            #print "AAAA  =>", artistA["_id"]
            #print "AAAA slug  =>",slugify(artistA["_id"]).lower()
            
            for artistB in artists.find({},no_cursor_timeout=True) :
                if artistA["_id"]==artistB["_id"]:
                    pass
                elif (fuzz.ratio(\
                                        slugify(artistA["_id"]).lower()\
                                        , slugify(artistB["_id"]).lower()\
                                       ) > FUZZ_RATIO and len(artistA["gigs"]) >= len(artistB["gigs"])) :
                    print "A|", artistA["_id"],"|",slugify(artistA["_id"]).lower(),"|",len(artistA["gigs"]),"|",artistA["mbid"],"|"
                    print "B|", artistB["_id"],"|",slugify(artistB["_id"]).lower(),"|",len(artistB["gigs"]),"|",artistB["mbid"],"|"
                    print "_______________________________"
                    for gig in artistB["gigs"]:
                        artistA["gigs"].append(gig)
                        
                    Aprime = artistA["gigs"]
                    #print Aprime
                    Aprime2 = sorted(Aprime,key=lambda t: t["datetime"].strftime(DATETIME_FORMAT))
                    #print "Aprime2",Aprime2
                    #BYPASS THE FOLLOWING LINE  WHEN THE SCRIPT IS MATURE
                    #Answer_art= query_yes_no("MERGE "+artistB["_id"]+ " WITH "+ artistA["_id"]+" ?" , default="no")
                    #print Answer_art
                    Answer_art = True
                    if Answer_art == True :
                        artistA["gigs"]=Aprime2
                        #print artistA["gigs"]
                        replacement_list.append([artistA["_id"],artistB["_id"],artistA])
                        
print "DONE!"
for  replacement in replacement_list:
    print replacement
    alias= replacement[1] 
    replacement[2]["alias"].append(alias)
    #artists.update({"_id" :replacement[0]}, replacement[2], upsert=True, multi=False)
    #artists.remove({"_id" :replacement[1]})

In [ ]:
from slugify import slugify
CSVDATA=[]
res = artists.find({})
print "NB OCCURENCES:",res.explain()['executionStats']["nReturned"]
COUNTER=0
for rep in res:
    COUNTER+=1
    a=str(rep.get('_id').encode('utf-8'))
    #print a
    CSVDATA.append([str(COUNTER),slugify(a.lower()),a])
        #print re['_id']
        #print CSVDATA 
    #    if len(CSVDATA>1:
    #           if (fuzz.ratio(re['_id'].lower(), CSVDATA[len(CSVDATA)]) >= 30) :
    #                print re['_id']
    #                CSVDATA.append(["\""+re['_id']+"\""])
    #        #CSVDATA.append([re['_id']])
     
print COUNTER
print CSVDATA[:5]

In [ ]:
from operator import itemgetter 
CSVDATA2=sorted(CSVDATA ,key =itemgetter(1)) 
print CSVDATA2[20:120]

In [ ]:
rowOLD=None
CNT=0
liste_doublons=[]
for row in CSVDATA2:
    #print row[1]
    if  rowOLD is not None and (fuzz.ratio(row[1], rowOLD[1]) >= 95) :
        print row ,rowOLD
        
        liste_doublons.append([row ,rowOLD])
        rowOLD=row
        CNT+=1
        
    if rowOLD is  None:
        
        rowOLD=row
    else:
        rowOLD=row
print len(CSVDATA2)
print CNT
print liste_doublons


In [ ]:
print liste_doublons[:5]
import csv
 
 
myFile = open('doublons_artistes_possibles.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(liste_doublons)
     
print("Writing complete")


In [ ]:
import csv
 
liste_doublonsTRIEE = []
with open('doublons_artistes_possiblesTRIES.csv') as File:
    reader = csv.DictReader(File)
    for row in reader:
        #print row
        
        aa=eval(row["ARTISTA"].replace('\"', ''))
        bb=eval(row["ARTISTB"].replace('\"', ''))
        liste_doublonsTRIEE.append([aa,bb])
        print liste_doublonsTRIEE[:3]

In [ ]:

       
for  paire in liste_doublonsTRIEE:
    print paire
    #print paire[0][2]
    winner=None
    looser=None
    for artistA in artists.find({"_id":paire[0][2]}):
        print artistA["mbid"]
        for artistB in artists.find({"_id":paire[1][2]}):
            pass
        #   print artistB["mbid"]
    
            
    if artistA["mbid"] is None and artistB["mbid"] is None :
        #check  gigs count return winner

        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    
        
    elif artistA["mbid"] is None and artistB["mbid"] is not None :
        #ARTISTB WINS
        winner=artistB
        looser=artistA
    elif artistA["mbid"] is not None and artistB["mbid"] is  None:
        #ARTIST A WINS
        winner=artistA
        looser=artistB
    elif artistA["mbid"] is not None and artistB["mbid"] is  not None:
         #check  gigs count nd return winner
        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    #apply modifications
    print "winner is", winner["_id"]
    print "looser is", looser["_id"]
    
    #for gig in looser["gigs"]:
     #   print gig
       # for venue in venues.find({"name": gig["venue"]["name"]  }):
        #    print venue
            
    #print winner["gigs"][0]
    for gig in looser["gigs"]:
        
        winner["gigs"].append(gig)
    print "========"
    #artists.update({"_id" :winner["_id"]}, winner, upsert=True, multi=False)
    
    #artists.remove({"_id" :looser["_id"]})
    print winner["gigs"]

In [ ]:
#FOR MANUAL EXTRAS

import csv
import codecs


liste_doublonsTRIEE = []


with open('doublons_artistes_possiblesTRIES2.csv') as File:
    reader = csv.DictReader(File)
    for row in reader:
        #print row
        
        aa=row["ARTISTA"]
        bb=row["ARTISTB"]
        liste_doublonsTRIEE.append([aa,bb])
    print liste_doublonsTRIEE

# -*- coding: utf-8 -*-
       
for  paire in liste_doublonsTRIEE:
    print paire[0]
    print paire[1]
    

In [ ]:

       
for  paire in liste_doublonsTRIEE:
    print paire
    #print paire[0][2]
    winner=None
    looser=None
    for artistA in artists.find({"_id":paire[0]}):
        print artistA["mbid"]
        for artistB in artists.find({"_id":paire[1]}):
            pass
        #   print artistB["mbid"]
    
            
    if artistA["mbid"] is None and artistB["mbid"] is None :
        #check  gigs count return winner

        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    
        
    elif artistA["mbid"] is None and artistB["mbid"] is not None :
        #ARTISTB WINS
        winner=artistB
        looser=artistA
    elif artistA["mbid"] is not None and artistB["mbid"] is  None:
        #ARTIST A WINS
        winner=artistA
        looser=artistB
    elif artistA["mbid"] is not None and artistB["mbid"] is  not None:
         #check  gigs count nd return winner
        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    #apply modifications
    print "winner is", winner["_id"]
    print "looser is", looser["_id"]
    
    #for gig in looser["gigs"]:
     #   print gig
       # for venue in venues.find({"name": gig["venue"]["name"]  }):
        #    print venue
            
    #print winner["gigs"][0]
    for gig in looser["gigs"]:
        
        winner["gigs"].append(gig)
    print "========"
    artists.update({"_id" :winner["_id"]}, winner, upsert=True, multi=False)
    
    artists.remove({"_id" :looser["_id"]})
    #print winner["gigs"]

In [ ]:
  #TIESTO /INGROSSO FIX


In [117]:
winner=None
looser=None
for artistA in artists.find({"_id":"Wayne Shorter"}):
        print artistA["mbid"]
        for artistB in artists.find({"_id":"Wayne Shorter Quartet"}):
            pass
        #   print artistB["mbid"]
if artistA["_id"] == artistB["_id"] :
    print "ERROR"
    
else:
    
            
 if artistA["mbid"] is None and artistB["mbid"] is None :
        #check  gigs count return winner

        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    
        
 elif artistA["mbid"] is None and artistB["mbid"] is not None :
        #ARTISTB WINS
        winner=artistB
        looser=artistA
 elif artistA["mbid"] is not None and artistB["mbid"] is  None:
        #ARTIST A WINS
        winner=artistA
        looser=artistB
 elif artistA["mbid"] is not None and artistB["mbid"] is  not None:
         #check  gigs count nd return winner
        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    #apply modifications
 print "winner is", winner["_id"]
 print "looser is", looser["_id"]
    
    #for gig in looser["gigs"]:
     #   print gig
       # for venue in venues.find({"name": gig["venue"]["name"]  }):
        #    print venue
            
    #print winner["gigs"][0]
 for gig in looser["gigs"]:
        
        winner["gigs"].append(gig)
 print "========"
 artists.update({"_id" :winner["_id"]}, winner, upsert=True, multi=False)
    
 artists.remove({"_id" :looser["_id"]})
    #print winner["gigs"]

2379937f-6e0d-46a2-b8ff-633fafd72002
winner is Wayne Shorter
looser is Wayne Shorter Quartet


/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:59: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:61: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


In [ ]:
import re
winner=None
looser=None
for artistA in artists.find({ "_id" :{ "$regex": ".* & .*"}  }):
        #print artistA["_id"].split("&")[0]
        for artistB in artists.find({ "_id" : artistA["_id"].split(" & ")[0]}):
            #print artistB["_id"]
            #print artistA["_id"]
            
            if artistA["mbid"] is None and artistB["mbid"] is None :
            #check  gigs count return winner
            
             if len(artistA["gigs"])>=len(artistB["gigs"]) :
              winner=artistA
              looser=artistB
             else:
              winner=artistB
              looser=artistA
              
              
              
              
            elif artistA["mbid"] is None and artistB["mbid"] is not None :
              #ARTISTB WINS
              winner=artistB
              looser=artistA
            elif artistA["mbid"] is not None and artistB["mbid"] is  None:
              #ARTIST A WINS
              winner=artistA
              looser=artistB
            elif artistA["mbid"] is not None and artistB["mbid"] is  not None:
              #check  gigs count nd return winner
              if len(artistA["gigs"])>=len(artistB["gigs"]) :
                  winner=artistA
                  looser=artistB
              else:
                  winner=artistB
                  looser=artistA
              
              
              #apply modifications
            print "winner is", winner["_id"]
            print "looser is", looser["_id"]
              
              #for gig in looser["gigs"]:
              #   print gig
              # for venue in venues.find({"name": gig["venue"]["name"]  }):
              #    print venue
              
              #print winner["gigs"][0]
            for gig in looser["gigs"]:
              
               winner["gigs"].append(gig)
            print "========"
            REP= query_yes_no("REPLACE",default="no")
            #REP=True
            #print REP
            if REP== True:
                print "rep"
                #artists.update({"_id" :winner["_id"]}, winner, upsert=True, multi=False)
              
                #artists.remove({"_id" :looser["_id"]})
            #print winner["gigs"]
        

In [108]:
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
winner=None
looser=None
RES=[]
for artist in artists.find({}):

       RES.append(artist["_id"])

In [109]:
    
i = 1
artistDOUBposs= []
while i < len(RES)-1:
    #print RES[i]
    
 if (RES[i]+" ") in RES[i+1] :
        
        print RES[i] , RES[i+1]
        artistDOUBposs.append([RES[i], RES[i+1]])
        
 i+=1
        

    

1.21 1.21 Gigawatts
10 Years 10 Years After, The Yardbirds, Henry's Funeral Shoe, The Graveltones, The Yardbirds,
100 Proof 100 Proof Hatred
100% 100% PRESI BENE
10th Letter 10th Letter Ensemble
12 12 Gauge Awakening
15 15 TIMES 4 MICKAEL BERTHELEMY piano, clavier SOFIAN EL MABROUK basse ALEX GAUTHIER batterie BENJAMIN BOUTANT saxophone
16 16 Candles
17 17 Crash
187 187 Strassenbande
2 Tone 2 Tone Kings 
20/20 20/20 Hindsight
21 21 Days Remain
22 22 Top
23 23 Angles of Attack
3 3 Are Legend
30 30 & Over League
311 311 Steel Pulse Jemere Morgan
33 33 AD
3D 3D FRIENDS
3rd 3rd Denial
4th 4th 'N Goal
50 50 Breaks
5am 5am At The Funeral Home
60 60 Grit
666 666 PACKS
70's 70's TUSH
8 Track 8 Track Cadillac
80 80 Doppel D
808 808 State
90 Proof 90 Proof Therapist
90's 90's Television
96 96 Olympics
A A  Veil Apart
A-Town A-Town GetDown
A.D.D. A.D.D. Tel-Aviv
A1 A1 Bassline
ADDI ADDI HUBER
AG AG Levine
AIR AIR RAID
AK AK KItten
AL AL DO IT LIKE THIS
ALO ALO WALA
AM AM Adam
AV AV AV AV
Aaron Aa

Daniel Avery Daniel Avery b2b Roman Flügel
Daniel Carter Daniel Carter alto saxophone
Daniel Dreier Daniel Dreier / Turmspringer
Daniel Martin Daniel Martin Moore
Daniel Sadownick Daniel Sadownick percussion
Danielle Reich Danielle Reich Music
Dank Dank 1
Danko Danko Jones
Danny Danny & The Tramp
Danny B. Harvey Danny B. Harvey Express
Danny Thompson Danny Thompson Deluxe
Danya Danya Lynn
Daphne Daphne Culver
Dara Tucker Dara Tucker Music
Darcy Darcy Dawn & Company
Dare Dare Defy
Daria Daria Kulesh
Darius Darius & Finlay
Dark Heart Dark Heart News
Dark Sky Dark Sky Choir
Dark Sun Dark Sun Bright
Dark White Dark White Light
Darkhorse Darkhorse Saloon
Darko Darko Brooker
Darlene Darlene Love
Darling Darling Bonnie
Darlington Darlington Pairs
Darrell Scott Darrell Scott Bluegrass Band
Darrow Darrow Chemical Company
Darwin Darwin Deez
Dash Dash Berlin
Data Data & Pata
Daughters Daughters of Davis
Daun Daun Giventi
Dave Dave  Vacant
Dave Adkins Dave Adkins Band
Dave Britt Dave Britt Band
Da

Heaven Heaven 17 - BEF
Heavix Heavix & Stickup
Heavyweight Heavyweight DJ's
Heck Heck Tate
Hectic Hectic Hobo
Hector Hector Acosta
Hedley Hedley Grange
Heed Heed The Assailant
Heejaz Heejaz Extended
Heel Heel Of Achilles
Heels Heels over Head
Heidi Heidi Baker 
Heights Heights Funk Collective
Heirs Heirs of Zigardy
Helen Helen Avakian
Helena Helena Deland
Helio Helio Alvez
Helix Helix Nebula
Hell City Hell City Kitty
Hell Fire Hell Fire Jack
Hellion Hellion Rising
Hello Hello Again
Helmut Helmut Cool
Heloise Heloise & Savoir Faire
Helvetica Helvetica Effect
Hemingway Hemingway Hero
Hemmingway Hemmingway Hammers
Henrietta Henrietta Lacks
Henry Henry Adams
Her Her & the Hymns
Herb Herb & Hanson
Herbie Herbie Hancock
Here Here & Sea
Heretic Heretic A.D.
Hero Hero & Villain
Heroes Heroes & Hypocrites
Hertz Hertz Donut
Hex Hex Campaign
Hey Zeus Hey Zeus Chrysler
Hezekiah Hezekiah Jones
Hidden Hidden Agenda
Hiding Hiding Out
High High 5 Hooray
High Five High Five Swan Dive
High Roller High R

Michael Blake Red Hook Soul Michael Blake Red Hook Soul CD Release
Michael Davis Michael Davis w Lions
Michael Glabicki Michael Glabicki of Rusted Root
Michael James Michael James Mette
Michael Janisch Michael Janisch "Paradigm Shift"
Michael Jr. Michael Jr. Comedy
Michael Martin Michael Martin Murphey
Michael Martin Murphey Michael Martin Murphey Fan Page
Michael Nau Michael Nau of Cotton Jones
Michael Olivera Michael Olivera on drums
Michael Powers Michael Powers Frequency
Michael Schenker Michael Schenker Group
Michael Sheppard Michael Sheppard Group
Michal Angela Michal Angela Wilson
Michelangelo Michelangelo Buzzi
Michelle Walker Michelle Walker Music
Michigan Michigan & Smiley
Mick Mick Boogie
Mickey Mickey 3D
Micky and Gary Braun Micky and Gary Braun to open!
Microwave Microwave Meatflowers
Midas 104 Midas 104 LIVE
Middle Finger Middle Finger Salute
Midlake Midlake Band
Midland Midland City
Midnight Midnight Avenue
Midnight North Midnight North feat. Grahame Lesh
Midnite Midnite

Saul Saul Conrad
Sausage Sausage Fingers
Savage Savage Annihilation
Savanah Savanah Harris
Savanna Savanna Leigh Bassett
Savannah Savannah Afros
Savi Savi Fernandez
Savi Fernandez Savi Fernandez Band
Savoy Savoy Brown
Sawtooth Sawtooth Tigers
Sax Sax Appeal
Saxon Saxon Lee Aaron
Say It Say It Ain't Soul
Sayer Sayer N Slim
Scala Scala & Kolacny Brothers
Scan Scan Hopper
Scanner Scanner Darkly
Scarecrow Scarecrow Electric
Scarlett Scarlett Avenue
Scarred Scarred By Beauty
Scars Scars Like MINE
Scavenger Scavenger Hunt
Scene Scene of Irony
Scenic Scenic Byway
Schizo Schizo Boy-DJ Team
Scholars Scholars Word
School of Rock School of Rock Evanston Tribute Shows
Schwarz Schwarz Don't Crack
Scooter Scooter Brown
Scooter Brown Scooter Brown Band
Scope Scope & Figure
Scorpio Scorpio Brothers
Scorpion Scorpion Child
Scotch Scotch & Sofa
Scott Scott & Charlene's Wedding
Scott Amendola Scott Amendola Band
Scott Hand Scott Hand Band
Scott Law Scott Law Acoustic MacHine
Scott McMahan Scott McMahan F

Trigger Trigger Creek
Triggers Triggers & Slips
Trina Trina Hamlin
Trine Trine Rein
Trini Trini Garza
Trinidad Trinidad Doherty
Trinity Trinity Bradshaw
Trio Trio 3
Trio 3 Trio 3 w/ Vijay Iyer
Tripwire Tripwire Fargo
Trisha Trisha Yearwood
Tristan Tristan Blaine
Tristen Tristen Brooke
Triton Triton Taylor
Triumph Triumph Over Shipwreck
Triumphant Triumphant Quartet
Trivia Trivia night
Troll Troll 2
Trolley Trolley Snatcha
Trouble Trouble & Daughter
Troy Redfern Troy Redfern Band
Troy Redfern Band Troy Redfern Band feat. Tosh Murase on drums
Truck Stop Truck Stop Gamblers
Trucker Trucker Dad
Trust Trust Circle
Trust Me Trust Me I Will
Truth Truth & Consequence
Try Try the Pie
Tsunami Tsunami Bomb
Tube Tube & Berger
Tuck Tuck & Patti
Tuff Tuff Culture
Tula Tula Vera
Tumbleweed Tumbleweed Company
Tumbleweed Wanderers Tumbleweed Wanderers to Open the show
Tunnel Tunnel Vision
Turbo Turbo Controller
Turetsky Choir Turetsky Choir Art Group
Turf Turf War
Turn Turn Around Jumper
Turquoise Jeep

In [110]:
print len(artistDOUBposs)

for  paire in artistDOUBposs:
    if paire == [u'Burial Ground', u'Burial Grounds'] :
        print artistDOUBposs.index([u'Burial Ground', u'Burial Grounds'])
        
        
["trombone","music","vocals","dj set","trio"," w/","- drums","- keys","- clarinette","Live"," chant "]

7935


['trombone',
 'music',
 'vocals',
 'dj set',
 'trio',
 ' w/',
 '- drums',
 '- keys',
 '- clarinette',
 'Live',
 ' chant ']

In [105]:
oo=0
for  paire in artistDOUBposs:
    #print paire[0]
    #print paire[1]
    ROSE= (paire[1].strip(paire[0])).lower()
    #for a in [" + ","& the","and the"]:
    #for a in [" band","quartet","duo","orchestra","trio","quintet","official","solo","(", "with"]:
    #for a in [" - "]:
    #for a in ["trombone","music","vocals","dj set","trio"," w/","- drums","- keys","- clarinette","Live"," chant "," band","featuring"]:    
    for a in ["-live-"]: 
     if a in ROSE:
        print "GO"
        print paire[0]
        print paire[1]
        

GO
Johnny Allwess
Johnny Allwess -live-
GO
Minimal Lounge
Minimal Lounge -live-
GO
Patrick Arbez
Patrick Arbez -live-
GO
Raumakustik
Raumakustik -live-


In [107]:
for  paire in artistDOUBposs:
    #print paire[0]
    #print paire[1]
 ROSE= (paire[1].strip(paire[0])).lower()
 #for a in [" / "," : "," + ","& the","and the"]:
 #for a in [" band","quartet","duo","orchestra","trio","quintet","official","solo","(", "with"," - "]:
 #for a in ["trombone","music","vocals","dj set","trio"," w/","- drums","- keys","- clarinette","Live"," chant "]:
 #for a in ["trombone","music","vocals","dj set"," trio "," w/","- drums","- keys","- clarinette","Live"," chant "," band","featuring"]:
 for a in ["-live-","band"]: 
  if a in ROSE:
    

    print paire
    #print paire[0][2]
    winner=None
    looser=None
    for artistA in artists.find({"_id":paire[0]}):
        print artistA["mbid"]
        for artistB in artists.find({"_id":paire[1]}):
            pass
        #   print artistB["mbid"]
    
            
    if artistA["mbid"] is None and artistB["mbid"] is None :
        #check  gigs count return winner

        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    
        
    elif artistA["mbid"] is None and artistB["mbid"] is not None :
        #ARTISTB WINS
        winner=artistB
        looser=artistA
    elif artistA["mbid"] is not None and artistB["mbid"] is  None:
        #ARTIST A WINS
        winner=artistA
        looser=artistB
    elif artistA["mbid"] is not None and artistB["mbid"] is  not None:
         #check  gigs count nd return winner
        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    #apply modifications
    print "winner is", winner["_id"]
    print "looser is", looser["_id"]
    
    #for gig in looser["gigs"]:
     #   print gig
       # for venue in venues.find({"name": gig["venue"]["name"]  }):
        #    print venue
    REP= query_yes_no("REPLACE",default="no")
    #REP=True
            #REP=True
            #print REP
    if REP== True:
            
    #print winner["gigs"][0]
     for gig in looser["gigs"]:
        
        winner["gigs"].append(gig)
     
     artists.update({"_id" :winner["_id"]}, winner, upsert=True, multi=False)
    
     artists.remove({"_id" :looser["_id"]})
    print "========"
    #print winner["gigs"]
              
 

[u'187', u'187 Strassenbande']
53b2acae-6757-4f47-be3b-4aa838a90af5
winner is 187
looser is 187 Strassenbande
REPLACE [y/N] 
[u'Blackberry Bushes', u'Blackberry Bushes Stringband']
None
winner is Blackberry Bushes
looser is Blackberry Bushes Stringband
REPLACE [y/N] y


/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:74: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:76: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


[u'Cali Shaw', u'Cali Shaw Band']
None
winner is Cali Shaw
looser is Cali Shaw Band
REPLACE [y/N] y
[u'Caroline Aiken', u'Caroline Aiken Band']
1c4bb57b-f36d-4aa5-8932-14cdc4c5a101
winner is Caroline Aiken
looser is Caroline Aiken Band
REPLACE [y/N] y
[u'Caroline Keller', u'Caroline Keller Band']
None
winner is Caroline Keller Band
looser is Caroline Keller
REPLACE [y/N] y
[u'Carvin Jones', u'Carvin Jones Band']
None
winner is Carvin Jones
looser is Carvin Jones Band
REPLACE [y/N] y
[u'Casey Daniels', u'Casey Daniels Band']
None
winner is Casey Daniels Band
looser is Casey Daniels
REPLACE [y/N] y
[u'Cass McCombs', u'Cass McCombs Band']
c213f661-70a8-42aa-b858-62f5bd2d72e9
winner is Cass McCombs
looser is Cass McCombs Band
REPLACE [y/N] y
[u'Charmaine Neville', u'Charmaine Neville Band']
08482e35-e32e-4938-8a5c-4d2bd07bfabc
winner is Charmaine Neville Band
looser is Charmaine Neville
REPLACE [y/N] y
[u'Chase Tyler', u'Chase Tyler Band']
None
winner is Chase Tyler Band
looser is Chase Ty

REPLACE [y/N] y
[u'Gerry Williams', u'Gerry Williams Band']
None
winner is Gerry Williams
looser is Gerry Williams Band
REPLACE [y/N] y
[u'Gipsy Moon', u'Gipsy Moon Band']
None
winner is Gipsy Moon
looser is Gipsy Moon Band
REPLACE [y/N] y
[u'Grace Pettis', u'Grace Pettis Band']
None
winner is Grace Pettis
looser is Grace Pettis Band
REPLACE [y/N] y
[u'Grayson Capps', u'Grayson Capps Band']
f2ab6fbd-e2d6-4f14-bc02-5dca733b2ed8
winner is Grayson Capps
looser is Grayson Capps Band
REPLACE [y/N] y
[u'Greg Horne', u'Greg Horne Band']
None
winner is Greg Horne
looser is Greg Horne Band
REPLACE [y/N] y
[u'Greg Kihn', u'Greg Kihn Band']
17b041e5-cc75-4b3e-9fd4-d599e15ddd0b
winner is Greg Kihn
looser is Greg Kihn Band
REPLACE [y/N] y
[u'Guy Forsyth', u'Guy Forsyth Band']
dc1b06cb-45f1-48ca-bee7-397de796fb02
winner is Guy Forsyth
looser is Guy Forsyth Band
REPLACE [y/N] y
[u'Haley Cole', u'Haley Cole Band']
None
winner is Haley Cole
looser is Haley Cole Band
REPLACE [y/N] y
[u'Hanne Mette', u'H

REPLACE [y/N] y
[u'Jon Lewis', u'Jon Lewis Band']
28b909a5-2f11-4b44-b8ff-039cdd3712c7
winner is Jon Lewis
looser is Jon Lewis Band
REPLACE [y/N] y
[u'Jonathan Mitchell', u'Jonathan Mitchell Band']
None
winner is Jonathan Mitchell
looser is Jonathan Mitchell Band
REPLACE [y/N] y
[u'Joseph King', u'Joseph King Band']
None
winner is Joseph King
looser is Joseph King Band
REPLACE [y/N] y
[u'Joseph Marcinek', u'Joseph Marcinek Band']
None
winner is Joseph Marcinek
looser is Joseph Marcinek Band
REPLACE [y/N] y
[u'Josh Arbour', u'Josh Arbour Band']
None
winner is Josh Arbour
looser is Josh Arbour Band
REPLACE [y/N] y
[u'Josh Fuller', u'Josh Fuller Band']
None
winner is Josh Fuller
looser is Josh Fuller Band
REPLACE [y/N] 
[u'Josh Garrett', u'Josh Garrett Band']
None
winner is Josh Garrett
looser is Josh Garrett Band
REPLACE [y/N] y
[u'Josh Moore', u'Josh Moore Band']
8d5bc063-9045-4f58-9edf-9f77a2d2dc6a
winner is Josh Moore
looser is Josh Moore Band
REPLACE [y/N] y
[u'Josh Netsky', u'Josh N

REPLACE [y/N] y
[u'Mike Stanley', u'Mike Stanley Band']
None
winner is Mike Stanley Band
looser is Mike Stanley
REPLACE [y/N] y
[u'Minimal Lounge', u'Minimal Lounge -live-']
None
winner is Mike Stanley Band
looser is Minimal Lounge
REPLACE [y/N] y
[u'Mojo Perry', u'Mojo Perry Band']
None
winner is Mojo Perry
looser is Mojo Perry Band
REPLACE [y/N] y
[u'Morgan Leigh', u'Morgan Leigh Band']
None
winner is Morgan Leigh
looser is Morgan Leigh Band
REPLACE [y/N] y
[u'Moses Jones', u'Moses Jones Band']
None
winner is Moses Jones
looser is Moses Jones Band
REPLACE [y/N] y
[u'Mountain Faith', u'Mountain Faith Band Concert']
None
winner is Mountain Faith
looser is Mountain Faith Band Concert
REPLACE [y/N] y
[u'Munk Duane', u'Munk Duane Band']
None
winner is Munk Duane Band
looser is Munk Duane
REPLACE [y/N] y
[u'Nashon Holloway', u'Nashon Holloway Band']
None
winner is Nashon Holloway
looser is Nashon Holloway Band
REPLACE [y/N] y
[u'Nat Osborn', u'Nat Osborn Band']
None
winner is Nat Osborn Ba

REPLACE [y/N] y
[u'Scott Pemberton', u'Scott Pemberton Band']
None
winner is Scott Pemberton
looser is Scott Pemberton Band
REPLACE [y/N] y
[u'Scottie Miller', u'Scottie Miller Band']
None
winner is Scottie Miller Band
looser is Scottie Miller
REPLACE [y/N] y
[u'Scotty Mac', u'Scotty Mac Band']
None
winner is Scotty Mac
looser is Scotty Mac Band
REPLACE [y/N] y
[u'Seth Moore', u'Seth Moore the Band']
None
winner is Seth Moore
looser is Seth Moore the Band
REPLACE [y/N] y
[u'Shane Martin', u'Shane Martin Band']
336355c2-05cc-44ae-b45c-3293ba102907
winner is Shane Martin
looser is Shane Martin Band
REPLACE [y/N] y
[u'Shane Rogers', u'Shane Rogers Band']
None
winner is Shane Rogers
looser is Shane Rogers Band
REPLACE [y/N] y
[u'Shannon Curfman', u'Shannon Curfman Band']
c2c6f558-c80a-4fb9-a7b7-fa308439af2e
winner is Shannon Curfman
looser is Shannon Curfman Band
REPLACE [y/N] y
[u'Shari Puorto', u'Shari Puorto Band']
None
winner is Shari Puorto Band
looser is Shari Puorto
REPLACE [y/N] y


REPLACE [y/N] y
[u'Vickie Vaughn', u'Vickie Vaughn Band']
None
winner is Vickie Vaughn
looser is Vickie Vaughn Band
REPLACE [y/N] y
[u'Victor Wooten', u'Victor Wooten Band']
1d0ae4e0-384f-4854-a1b4-ebb4cc81414b
winner is Victor Wooten
looser is Victor Wooten Band
REPLACE [y/N] y
[u'Vince Neil', u'Vince Neil Band']
ce39ec95-5476-4c65-a862-fec96ff851ec
winner is Vince Neil
looser is Vince Neil Band
REPLACE [y/N] y
[u'Virginia Rose', u'Virginia Rose Band']
None
winner is Virginia Rose
looser is Virginia Rose Band
REPLACE [y/N] y
[u'Warren Haynes', u'Warren Haynes Band']
4b3b9c7e-c32e-4e62-bbdb-d2e8aaffe66e
winner is Warren Haynes
looser is Warren Haynes Band
REPLACE [y/N] y
[u'Wayne Garner', u'Wayne Garner Band']
None
winner is Wayne Garner Band
looser is Wayne Garner
REPLACE [y/N] y
[u'Wes Nickson', u'Wes Nickson Band']
None
winner is Wes Nickson Band
looser is Wes Nickson
REPLACE [y/N] y
[u'Wes Perryman', u'Wes Perryman Band']
None
winner is Wes Perryman
looser is Wes Perryman Band
REPL

In [23]:

#i=0
#for  paire in artistDOUBposs:
# if i <121:
    
 #   
        
  #      i+=1
   #     pass
# else :
 
    
    
   
    print paire
    #print paire[0][2]
    winner=None
    looser=None
    for artistA in artists.find({"_id":paire[0]}):
        print artistA["mbid"]
        for artistB in artists.find({"_id":paire[1]}):
            pass
        #   print artistB["mbid"]
    
            
    if artistA["mbid"] is None and artistB["mbid"] is None :
        #check  gigs count return winner

        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    
        
    elif artistA["mbid"] is None and artistB["mbid"] is not None :
        #ARTISTB WINS
        winner=artistB
        looser=artistA
    elif artistA["mbid"] is not None and artistB["mbid"] is  None:
        #ARTIST A WINS
        winner=artistA
        looser=artistB
    elif artistA["mbid"] is not None and artistB["mbid"] is  not None:
         #check  gigs count nd return winner
        if len(artistA["gigs"])>=len(artistB["gigs"]) :
            winner=artistA
            looser=artistB
        else:
             winner=artistB
             looser=artistA

            
    #apply modifications
    print "winner is", winner["_id"]
    print "looser is", looser["_id"]
    
    #for gig in looser["gigs"]:
     #   print gig
       # for venue in venues.find({"name": gig["venue"]["name"]  }):
        #    print venue
    REP= query_yes_no("REPLACE",default="no")
            #REP=True
            #print REP
    if REP== True:
            
    #print winner["gigs"][0]
     for gig in looser["gigs"]:
        
        winner["gigs"].append(gig)
     
     artists.update({"_id" :winner["_id"]}, winner, upsert=True, multi=False)
    
     artists.remove({"_id" :looser["_id"]})
    print "========"
    #print winner["gigs"]
              
 

[u'Absu', u'Absurd']
e4d0504d-a0cc-486e-9f8f-32da470698a5
winner is Absu
looser is Absurd
REPLACE [y/N] n
[u'Absurd', u'Absurd Bird']
48f6094f-24b2-4844-9d2e-2a2e87e51221
winner is Absurd
looser is Absurd Bird
REPLACE [y/N] n
[u'Abyssal', u'Abyssal Delirium']
None
winner is Abyssal
looser is Abyssal Delirium
REPLACE [y/N] n
[u'Acadia', u'Acadians']
None
winner is Acadians
looser is Acadia
REPLACE [y/N] n
[u'Accelerator', u'Accelerators']
None
winner is Accelerators
looser is Accelerator
REPLACE [y/N] n
[u'Accept', u'Acceptable Losses']
41f4d85a-0bd7-4602-a3e3-8c47f36efb0a
winner is Accept
looser is Acceptable Losses
REPLACE [y/N] n
[u'Access', u'Access Unlocked']
1d422f89-d724-4ec5-9e19-a24a1f46d67d
winner is Access
looser is Access Unlocked
REPLACE [y/N] n
[u'Accident', u'Accident Prone']
8a873164-16b0-4857-a2d1-e74842ef0877
winner is Accident
looser is Accident Prone
REPLACE [y/N] n
[u'Ace', u'Ace Alvarez']
56af1bdf-36b8-4816-8504-84896015b381
winner is Ace
looser is Ace Alvarez
REPL

/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:73: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/home/goonieb/gitrep/bandstour/venv/lib/python2.7/site-packages/ipykernel_launcher.py:75: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


[u'Acid Witch', u'Acid Witch, Nuclear Hellfrost, Sauron, Cavalcade ']
6574ecb6-1779-4a21-8678-a34d29dbfecb
winner is Acid Witch
looser is Acid Witch, Nuclear Hellfrost, Sauron, Cavalcade 
REPLACE [y/N] y
[u'Ackboo', u"Ackboo feat. S'Kaya"]
None
winner is Ackboo
looser is Ackboo feat. S'Kaya
REPLACE [y/N] y
[u'Acme', u'Acme Dynamite']
a0b63adb-58b0-476f-b3c4-66a34f52a061
winner is Acme
looser is Acme Dynamite
REPLACE [y/N] 
[u'AcoustiCafe', u'AcoustiCafe Band']
None
winner is AcoustiCafe
looser is AcoustiCafe Band
REPLACE [y/N] y
[u'Acoustic Jam', u'Acoustic Jams']
None
winner is Acoustic Jam
looser is Acoustic Jams
REPLACE [y/N] 
[u'Acoustic Trio', u'Acoustic Trio (Damien, Facs, Jack Paps)']
None
winner is Acoustic Trio
looser is Acoustic Trio (Damien, Facs, Jack Paps)
REPLACE [y/N] y
[u'Acoustic, Riley Green', u'Acoustic, Riley Green Muscadine Bloodline']
None
winner is Acoustic, Riley Green
looser is Acoustic, Riley Green Muscadine Bloodline
REPLACE [y/N] y
[u'Acrimonious', u'Acrimon

REPLACE [y/N] y
[u'Adversity', u'Adversity Breeds Malice']
None
winner is Adversity
looser is Adversity Breeds Malice
REPLACE [y/N] 
[u'Aeolia', u'Aeolian Race']
None
winner is Aeolia
looser is Aeolian Race
REPLACE [y/N] 
[u'Aeon', u'Aeon Patronist']
26e81e1e-1281-4485-83fe-08121c1cf4fb
winner is Aeon
looser is Aeon Patronist
REPLACE [y/N] 
[u'Aer', u'Aera (live)']
None
winner is Aer
looser is Aera (live)
REPLACE [y/N] 
[u'Aerial', u'Aerial East']
343e2b56-0d3c-4940-b06c-8ab7752637d8
winner is Aerial
looser is Aerial East
REPLACE [y/N] 
[u'Aeroplane', u'Aeroplane Flies High']
118c5949-3023-4262-a646-dda45e02f49d
winner is Aeroplane
looser is Aeroplane Flies High
REPLACE [y/N] 
[u'Affair', u'Affairs']
e7e0423b-1201-4222-9135-01fcf871f595
winner is Affair
looser is Affairs
REPLACE [y/N] 
[u'Affliction', u'Affliction Gate']
98d6bb97-a794-4ad5-b4ba-53e232e81316
winner is Affliction
looser is Affliction Gate
REPLACE [y/N] 
[u'Africa', u'Africa Entsha']
f0358093-c941-4c04-b68a-e6de12e61e32
w

REPLACE [y/N] 
[u'Aladin', u'Aladino']

winner is Aladin
looser is Aladino
REPLACE [y/N] 
[u'Alain Asplanato', u'Alain Asplanato , drums']
None
winner is Alain Asplanato
looser is Alain Asplanato , drums
REPLACE [y/N] y
[u'Alameda', u'Alamedadosoulna']
None
winner is Alamedadosoulna
looser is Alameda
REPLACE [y/N] 
[u'Alan Barnes', u'Alan Barnes Octet']
92db0f94-1c9f-4841-8baf-2b7ea823b20a
winner is Alan Barnes
looser is Alan Barnes Octet
REPLACE [y/N] y
[u'Alan Bennett', u'Alan Bennett Music']
None
winner is Alan Bennett Music
looser is Alan Bennett
REPLACE [y/N] y
[u'Alan Evans', u'Alan Evans Trio']
None
winner is Alan Evans
looser is Alan Evans Trio
REPLACE [y/N] y
[u'Alan Hall', u'Alan Hall - drum set']
None
winner is Alan Hall - drum set
looser is Alan Hall
REPLACE [y/N] y
[u'Alan Hall - drums', u'Alan Hall - drums (leader)']
None
winner is Alan Hall - drums
looser is Alan Hall - drums (leader)
REPLACE [y/N] y
[u'Alan Merril', u'Alan Merrill']
None
winner is Alan Merrill
looser is

...

REPLACE [y/N] 
[u'Body Count', u'Body Count ft. Ice-T']
5d365ed4-0a33-49ff-8de7-c0a47710eb31
winner is Body Count
looser is Body Count ft. Ice-T
REPLACE [y/N] y
[u'Body Language', u'Body Language (DJ SET)']
1a98f1f2-c2e1-4c3c-8ce5-c746979be489
winner is Body Language
looser is Body Language (DJ SET)
REPLACE [y/N] y
[u'Boh', u'Bohan Pheonix']
None
winner is Boh
looser is Bohan Pheonix
REPLACE [y/N] 
[u'Bohumil', u'Bohumil (Official)']
None
winner is Bohumil (Official)
looser is Bohumil
REPLACE [y/N] y
[u'Bolero', u'Boleros']
f37d75b9-560e-4e39-bc84-7bce97511994
winner is Bolero
looser is Boleros
REPLACE [y/N] 
[u'Bolt', u'Bolt Thrower']
b8322837-7248-481c-b221-d838f7a8fda3
winner is Bolt
looser is Bolt Thrower
REPLACE [y/N] 
[u'Bomb', u'Bomb Da Von']
abe29ea7-8237-4a8f-9146-c93c65ec92f9
winner is Bomb
looser is Bomb Da Von
REPLACE [y/N] 
[u'Bombay', u'Bombay Bicycle Club']
29bb8f9f-05f5-43ae-84b0-3b2d22303d40
winner is Bombay Bicycle Club
looser is Bombay
REPLACE [y/N] 
[u'Bombay Bicycl

[u'Bosque', u'Bosque Brown']
46b71b4e-956d-4fff-9a45-be8c6b1d05dc
winner is Bosque Brown
looser is Bosque
REPLACE [y/N] 
[u'Boss Axis', u'Boss Axis, Nico Pusch & Tom Pusch ...']
None
winner is Boss Axis
looser is Boss Axis, Nico Pusch & Tom Pusch ...
REPLACE [y/N] y
[u'Bosse', u'Bosse De Nage']
ba8fe46e-af81-4fcd-8581-59d6920d5d55
winner is Bosse
looser is Bosse De Nage
REPLACE [y/N] 
[u'Boston', u'Boston 168']
0d8b0d50-e4cf-4da4-965d-f24c58ec3268
winner is Boston
looser is Boston 168
REPLACE [y/N] 
[u'Boston Bun', u'Boston Bun (Ed Banger, FR)']
None
winner is Boston Bun
looser is Boston Bun (Ed Banger, FR)
REPLACE [y/N] y
[u'Boston Red Sox', u'Boston Red Sox at New York Yankees']
None
winner is Boston Red Sox
looser is Boston Red Sox at New York Yankees
REPLACE [y/N] y
[u'Bottle', u'Bottle Kids']
524fea86-8ce7-4643-ab38-5b13c6bb5729
winner is Bottle
looser is Bottle Kids
REPLACE [y/N] 
[u'Bottle Rocket', u'Bottle Rocket Science']
d6e64e36-b5e2-409a-a65e-44bda76f17ae
winner is Bottle R

REPLACE [y/N] y
[u'Briel', u'Brielle Batino']
None
winner is Briel
looser is Brielle Batino
REPLACE [y/N] 
[u'Brig', u'Briga']
None
winner is Briga
looser is Brig
REPLACE [y/N] 
[u'Briga', u'Brigade S']
None
winner is Briga
looser is Brigade S
REPLACE [y/N] 
[u'Bright', u'Bright At Night']
04f43a78-0e12-46c6-befb-afd691e66176
winner is Bright
looser is Bright At Night
REPLACE [y/N] 
[u'Brighton', u'Brighton Beat']
46f3c7dc-1e5c-4f96-9207-414ea5ee33ec
winner is Brighton
looser is Brighton Beat
REPLACE [y/N] 
[u'Brightside', u'Brightside (US)']
None
winner is Brightside (US)
looser is Brightside
REPLACE [y/N] y
[u'Brigitte', u'Brigitte Balma']
a7e2a75b-21c7-4bce-bf0c-a11748450b8e
winner is Brigitte
looser is Brigitte Balma
REPLACE [y/N] 
[u'Brigitte DeMeyer', u'Brigitte DeMeyer, Sara Petite, Ange Boxall']
None
winner is Brigitte DeMeyer
looser is Brigitte DeMeyer, Sara Petite, Ange Boxall
REPLACE [y/N] y
[u'Brill', u'Brilliant Beast']
None
winner is Brilliant Beast
looser is Brill
REPLAC

REPLACE [y/N] 
[u'Bryan Adams', u'Bryan Adams Experience']
4dbf5678-7a31-406a-abbe-232f8ac2cd63
winner is Bryan Adams
looser is Bryan Adams Experience
REPLACE [y/N] 
[u'Bryan Hayes', u'Bryan Hayes Hosts "Live At Farmhouse Studio"']
None
winner is Bryan Hayes
looser is Bryan Hayes Hosts "Live At Farmhouse Studio"
REPLACE [y/N] y
[u'Bryan May', u'Bryan Mayer']
None
winner is Bryan Mayer
looser is Bryan May
REPLACE [y/N] 
[u'Bryan Sutton', u'Bryan Sutton Band']
584f1bd9-01eb-4f7e-8695-2fa72bb7bf27
winner is Bryan Sutton
looser is Bryan Sutton Band
REPLACE [y/N] y
[u'Bryan White', u'Bryan White, David Ball']
e03c5f07-71e9-4e3c-a9fd-2a90e61d2f2e
winner is Bryan White
looser is Bryan White, David Ball
REPLACE [y/N] y
[u'Bryce Alber', u'Bryce Alber (aka Lou Jane)']
None
winner is Bryce Alber
looser is Bryce Alber (aka Lou Jane)
REPLACE [y/N] y
[u'Bryce Merrit', u'Bryce Merritt']
None
winner is Bryce Merritt
looser is Bryce Merrit
REPLACE [y/N] y
[u'Brynn Elliot', u'Brynn Elliott']
None
winner

KeyboardInterrupt: 

In [ ]:
# """" FOUND AT LHE LAST MINUTE WHILE RUNNING NODE/EDGE CREATOR
Aloe Blacc
Aloe Blacc w/ Quetzal

winner is Antares
looser is Antares UK

Antonella Ruggiero
looser is Antonella Ruggiero e Ensemble Hyperion


winner is Ash St. John Music
looser is Ash St. John


u'Belvedere', u'Belvedere, The Real Deal']
405ac86d-51f2-4dab-8066-06fccea33d9d
winner is Belvedere
looser is Belvedere, The Real Deal
REPLACE [y/N] 
==
========
[u'Ben Allen', u'Ben Allen Band']
4feb8396-ed6d-4907-9058-118bdec441b9
winner is Ben Allen
looser is Ben Allen Band

winner is Bitter Verses
looser is Bitter Verses (DE)

winner is Blizzard Of Oz
looser is Blizzard Of Ozz

winner is Bryan Adams
looser is Bryan Adams Experience